# 装饰器的各种实例

In [1]:
import time
 
def cached(function_to_decorate):
    _cache = {} # Where we keep the results
    def decorated_function(*args):
        start_time = time.time()
        print('_cache:', _cache)
        if args not in _cache:
            _cache[args] = function_to_decorate(*args) # Perform the computation and store it in cache
        print('Compute time: %ss' % round(time.time() - start_time, 2))
        return _cache[args]
    return decorated_function
 
@cached
def complex_computation(x, y):
    print('Processing ...')
    time.sleep(2)
    return x + y
 
print(complex_computation(1, 2)) # 3, Performing the expensive operation
print(complex_computation(1, 2)) # 3, SKIP performing the expensive operation
print(complex_computation(4, 5)) # 9, Performing the expensive operation
print(complex_computation(4, 5)) # 9, SKIP performing the expensive operation
print(complex_computation(1, 2)) # 3, SKIP performing the expensive operation

_cache: {}
Processing ...
Compute time: 2.0s
3
_cache: {(1, 2): 3}
Compute time: 0.0s
3
_cache: {(1, 2): 3}
Processing ...
Compute time: 2.0s
9
_cache: {(1, 2): 3, (4, 5): 9}
Compute time: 0.0s
9
_cache: {(1, 2): 3, (4, 5): 9}
Compute time: 0.0s
3


If you look at the code shallowly, you might object. The decorator isn't reusable! If we decorate another function (say another_complex_computation) and call it with the same parameters then we'll get the cached results from the complex_computation function. This won't happen. The decorator is reusable, and here's why:

In [2]:
@cached
def another_complex_computation(x, y):
    print('Processing ...')
    time.sleep(2)
    return x * y
     
print(another_complex_computation(1, 2)) # 2, Performing the expensive operation
print(another_complex_computation(1, 2)) # 2, SKIP performing the expensive operation
print(another_complex_computation(1, 2)) # 2, SKIP performing the expensive operation

_cache: {}
Processing ...
Compute time: 2.01s
2
_cache: {(1, 2): 2}
Compute time: 0.0s
2
_cache: {(1, 2): 2}
Compute time: 0.0s
2


The cached function is called once for every function it decorates, so a different _cache variable is instantiated every time and lives in that context. Let's test this out:




In [4]:
print(complex_computation(10, 20))           # -> 30
print(another_complex_computation(10, 20))   # -> 200

_cache: {(1, 2): 3, (4, 5): 9}
Processing ...
Compute time: 2.0s
30
_cache: {(1, 2): 2}
Processing ...
Compute time: 2.01s
200


The decorator we just coded, as you may have noticed, is very useful. It's so useful that a more complex and robust version already exists in the standard functools module. It is named lru_cache. LRU is the abbreviation of Least Recently Used, a caching strategy. 

In [5]:
from functools import lru_cache
 
@lru_cache()
def complex_computation(x, y):
    print('Processing ...')
    time.sleep(2)
    return x + y
 
print(complex_computation(1, 2)) # Processing ... 3
print(complex_computation(1, 2)) # 3
print(complex_computation(2, 3)) # Processing ... 5
print(complex_computation(1, 2)) # 3
print(complex_computation(2, 3)) # 5

Processing ...
3
3
Processing ...
5
3
5


In [6]:
from flask import Flask
 
app = Flask(__name__)
 
@app.route("/")
def hello():
    return "Hello World!"
 
if __name__ == "__main__":
    app.run()


ModuleNotFoundError: No module named 'flask'

The app.route decorator assigns the function hello as the request handler for the route "/". The simplicity is amazing. 

Another neat use of decorators is inside Django. Usually, web applications have two types of pages: 

pages you can view without being authenticated (front page, landing page, blog post, login, register)
pages you need to be authenticated to view (profile settings, inbox, dashboard)
If you try to view a page of the latter type, you'll usually get redirected to a login page. Here's how to implement that in Django:

In [7]:
from django.http import HttpResponse
from django.contrib.auth.decorators import login_required
 
# Public Pages
 
def home(request):
    return HttpResponse("<b>Home</b>")
 
def landing(request):
    return HttpResponse("<b>Landing</b>")
 
# Authenticated Pages
 
@login_required(login_url='/login')
def dashboard(request):
    return HttpResponse("<b>Dashboard</b>")
 
@login_required(login_url='/login')
def profile_settings(request):
    return HttpResponse("<b>Profile Settings</b>")

ModuleNotFoundError: No module named 'django'

Observe how neatly the private views are marked with login_required. While going through the code, it is very clear to the reader which pages require the user to log in and which pages do not.



In [9]:
def myFunction(n):                                                                                                        
  time.sleep(n) 

In [10]:
import time                                                                                                               
                                                                                                                          
def measure_time(func):                                                                                                   
                                                                                                                          
  def wrapper(*arg):                                                                                                      
      t = time.time()                                                                                                     
      res = func(*arg)                                                                                                    
      print("Function took " + str(time.time()-t) + " seconds to run")                                                    
      return res                                                                                                          
                                                                                                                          
  return wrapper                                                                                                          
                                                                                                                          
@measure_time                                                                                                             
def myFunction(n):                                                                                                        
  time.sleep(n)                                                                                                           
                                                                                                                          
if __name__ == "__main__":                                                                                                
    myFunction(2)     

Function took 2.0044000148773193 seconds to run
